Librerias

In [2]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.1.0


Lectura de dataset

Para la lectura del modelo necesitamos armar un dataset que contenga: 
CDD1 
   - Diffx 
   - Diffy
   - EventNumber
   - Angulo


x : [Diffx, Diffy] (filtrado por Eventos) 

y : [cos(Angulo),sin(Angulo)]

Pruebas: 
   1. Con la información de Diffx e Diffy tratar de identificar el angulo. (más rápido) 
   2. Reconstruir la imagen con Diffx e Diffy, reemplazar las capas iniciales por capas convolucionales. (más lento) 

In [ ]:
# Realizar la lectura de los datos y armar la esstructura dejar en x e y
# Directorios
path = 'C:/Users/Karina Aquino/Desktop/Tesis-main/roots/database/*'
directory = 'C:/Users/Karina Aquino/Desktop/Tesis-main/roots/database/'
s = "/"
ss = "/*"

# Se arma el dataframe 
columns = ['DiffX', 'DiffY', 'eventNumber','angle' , 'folder CCD', 'CCD'] # Aca se debe agregar las columnas que se deseea agregar al dataset 
data = {}  # Generar los datos genericos
for c in columns:
    data[c] = []    
df = pd.DataFrame(data)

# Se leen los datos y se carga en el dataset
files = glob.glob(path)
filesCount = len(files)
for doserank in files:
    dr = os.path.basename(os.path.normpath(doserank)) 
    doserank =  glob.glob(doserank + ss)
    for muonCCD in doserank:
        muonCCD1 = os.path.basename(os.path.normpath(muonCCD))
        muonCCD = glob.glob(muonCCD + ss)
        for CCD1 in muonCCD:
            if (muonCCD1 == "muonCCD1"):
                #print(CCD1)
                datos = glob.glob(os.path.normpath(CCD1)+ss)
                aux = pd.Series(( np.load(os.path.normpath(datos[0])),np.load(os.path.normpath(datos[1])), np.load(os.path.normpath(datos[2])), dr, muonCCD1), index=data)
                df = df.append(aux, ignore_index=True)

In [ ]:
# X = (DiffX,DiffY)
df_content = list()
X = list()
true_labels = list()
for index, row in df.iterrows():
    #xd = np.vstack((row['DiffX'][:], row['DiffY'][:], row['Z'][:], row['eventNumber'][:]))
    for i in range(len(row['DiffX'])):
        df_content.append([row['DiffX'][i], row['DiffY'][i], row['eventNumber'][i], row['angle'][i], row['folder CCD'][i], row['CCD'][i]])
        X.append([row['DiffX'][i], row['DiffY'][i]])
        true_labels.append(row['angle'][i])

Normalizar los datos

Separar en x_train, y_train , x_test,y_test 

Modelo

In [5]:
input_shape_model = 200
def build_model():
  model = keras.Sequential([
    
    #
      
    # Capas densas "normales"
    layers.Dense(64, activation='relu', input_shape=[input_shape_model]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(32,activation='relu'),
    layers.Dense(2)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                12864     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 66        
Total params: 27,490
Trainable params: 27,490
Non-trainable params: 0
____________________________________________________

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

Graficos de entrenamiento

Salida